In [1]:
import os
import pandas as pd
from openbabel import pybel
from freesasa import Structure, calc

In [2]:
PROTEIN_STRUC = "./1443.pdb"
LIGANDS_DIR = "./ligand"
COMPLEX_DIR = "./complex"
output_file = "./rsa_1443_select.csv"

In [ ]:
ligand_files = [f for f in os.listdir(LIGANDS_DIR) if f.endswith(('.pdbqt'))]
len(ligand_files)

In [ ]:
def build_complex(protein_file, ligand_file, output_file):
    try:
        protein = next(pybel.readfile("pdb", protein_file))
        ligand = next(pybel.readfile("pdbqt", ligand_file))

        complex_mol = pybel.ob.OBMol()
        complex_mol += protein.OBMol
        complex_mol += ligand.OBMol
        complex_pybel = pybel.Molecule(complex_mol)

        complex_pybel.write("pdb", output_file, overwrite=True)
        return True
    except Exception as e:
        print(f"Error building complex for {ligand_file}: {e}")
        return False

In [ ]:
results = []
failed = 0
success = 0
for i, ligand_file in enumerate(ligand_files):
    ligand_path = os.path.join(LIGANDS_DIR, ligand_file)
    ligand_name = os.path.splitext(ligand_file)[0]
    complex_pdb = os.path.join(COMPLEX_DIR, f"{ligand_name}_complex.pdb")

    print(f"processing ligand {i+1}/{len(ligand_files)}: {ligand_name}")
    build_complex(PROTEIN_STRUC, ligand_path, complex_pdb)
    
    try:
        structure = Structure(complex_pdb, options={"hetatm":True})
    except Exception as e:
        print(f"error reading complex structure for {ligand_name}: {e}")
        continue
    
    result = calc(structure)
    if result.residueAreas()['A']['1'].relativeTotal is None:
        print(f"failed to calculate RSA for {ligand_name}")
        failed = failed + 1
    else:
        results.append({
        'Ligand': ligand_name,
        'Total_RSA' : result.residueAreas()['A']['1'].relativeTotal,
        'Side_chain_RSA' : result.residueAreas()['A']['1'].relativeSideChain
        })
        success = success + 1
print(f"RSA calculation completed: {len(ligand_files)} total, {success} successful, {failed} failed.")


In [ ]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by = 'Side_chain_RSA', ascending=True)
results_df.columns = ['title_vina', 'Total_RSA', 'Side_chain_RSA']
results_df['title_vina'] = results_df['title_vina'] + ".pdbqt"

In [ ]:
df_map = pd.read_csv('./pharm_1443_select.csv')
rsa_select = df_map.merge(results_df.copy(), 
    on = 'title_vina',
    how='left',
    suffixes=('', '_dup'))
rsa_select

In [ ]:
#rsa_select['Side_chain_RSA'] = rsa_select['Side_chain_RSA'].to_numeric()

rsa_select = rsa_select[rsa_select['Side_chain_RSA']<=0.07]
rsa_select = rsa_select[rsa_select['Total_RSA']<=0.35]
rsa_select

In [ ]:
rsa_select.to_csv(output_file, index=False)